### GAN Art Creating Kaggle Project

In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import time

import tensorflow as tf
from tensorflow.keras import layers
import shutil

### Brief Description of the Problem and Data

The task in this competition is to create 7,000 to 10,000 pictures that Monet-style pictures. That is, the task is to create pictures that look like they could have been pained by Claude Monet and that a trained classifier cannot distinguish as fakes.

The competition comes with a dataset that consists of 300 Monet paintings (sized 256x256 in JPEG format) and 7028 photos (sized 256x256 in JPEG format). The aim is to use the Monet painting to train a GAN (Generative Adversarial Network) to master the style of Monet and use the GAN to style the pictures in a way that they look like they could have been pained by Monet. Alternatively, the GAN can simply generate images in the style of Monet without making use of the pictures, which is the approach we choose for this project.

The dataset also contains copies of the paintings and pictures in the TFRecords format. The TFRecords are supposed to be faster to process for tenosorflow and for that reason we will make use of them in our project.  

### Exploratory Data Analysis (EDA) — Inspect, Visualize and Clean the Data

In this section, we will ...
* verify that we can access all avialable files
* control that there are 300 paintings and 7028 photos
* visually inspect a few paintings and photos
* Comapare the color intensity of paintings and photos

### Utility functions

In [ ]:
def count_images_in_folder(folder_path):
    """
    Counts the number of image files in a given directory.
    
    Args:
    folder_path (str): The path to the directory containing images.
    
    Returns:
    int: The number of image files in the directory.
    """
    # List of image file extensions you want to check for
    image_extensions = ('.jpg', ) # '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'
    
    # Get a list of files in the directory
    files_in_folder = os.listdir(folder_path)
    
    # Count files that end with any of the image extensions
    image_count = sum(1 for file in files_in_folder if file.lower().endswith(image_extensions))
    
    return image_count

In [ ]:
def display_images(path):
    # List all JPEG files in the specified directory
    files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]
    
    # Select the first 9 images (or fewer if not enough images are available)
    files = files[:9]
    
    # Set up the figure and axes for a 3x3 grid
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    axes = axes.ravel()
    
    # Loop over the files and the axes
    for ax, img_path in zip(axes, files):
        # Open and display the image
        img = Image.open(img_path)
        ax.imshow(img)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(os.path.basename(img_path))
    
    # Hide any unused axes if there are less than 9 images
    for ax in axes[len(files):]:
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def compare_image_colors_normalized(path1, path2):
    """
    Compares the normalized average RGB values between two sets of images from different folders,
    using distinct color shades for better visual distinction.

    Args:
    path1 (str): Path to the first directory containing pictures.
    path2 (str): Path to the second directory containing paintings.
    """
    def get_avg_colors(image_folder):
        image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg'))]
        avg_colors = []
        for file in image_files:
            with Image.open(file) as img:
                img = img.convert('RGB')
                data = np.array(img)
                mean_color = data.mean(axis=(0, 1))
                avg_colors.append(mean_color)
        return np.array(avg_colors)
    
    # Get average colors for both directories
    avg_colors1 = get_avg_colors(path1)  # Pictures
    avg_colors2 = get_avg_colors(path2)  # Paintings
    
    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(8, 10), sharex=True)
    color_labels = ['Red', 'Green', 'Blue']
    dark_shades = ['darkred', 'darkgreen', 'darkblue']  # Darker shades for pictures
    pastel_shades = ['salmon', 'lightgreen', 'lightblue']  # Pastel shades for paintings
    
    for i in range(3):
        axes[i].hist(avg_colors1[:, i], bins=30, alpha=0.7, label='Pictures', color=dark_shades[i], density=True)
        axes[i].hist(avg_colors2[:, i], bins=30, alpha=0.7, label='Paintings', color=pastel_shades[i], density=True, linestyle='dashed')
        axes[i].set_title(f'Normalized Comparison of {color_labels[i]} Channel')
        axes[i].legend(loc='upper right')

    plt.tight_layout()
    plt.show()


### Problem Investigation

In [ ]:
# Overview of available files
Input = '/kaggle/input/gan-getting-started'
print(os.listdir(Input))

In [ ]:
# Count number of pictures
count_images_in_folder(Input + '/photo_jpg')

In [ ]:
# Count number of paintings
count_images_in_folder(Input + '/monet_jpg')

In [ ]:
# Photo
display_images(Input + '/photo_jpg')

In [ ]:
# Monet Paintings
display_images(Input + '/monet_jpg')

In [ ]:
compare_image_colors_normalized(Input + '/monet_jpg', Input + '/photo_jpg')

### Conclusion

Based on our investigations, we can conclude that:

* The dataset contains paintings and pictures in the formats jpg and TFrecords
* There are 300 paintings in the dataset as adverticed in the competition, but there are 7038 pictures, which are 10 more that adverticed.
* Visual inspection of images verfied that both the pictures and the paintings looked correct. 
* An analysis of the color intensity found that the pictures in average made use of more intense colors that the paintings. The paintings are a bit muted compared to the pictures

All in all, this implies that there are good reasons to believe that the dataset can be used to train a functioning GAN.

### Model Architecture

This project makes use of a Deep Convolutional Generative Adversarial Network (DCGAN) architecture. The architecture consists of two main pars:

The **Generator**, which takes a random noise vector as input and outputs a synthetic image. The generator tries to produce images that are indistinguishable from real images, effectively "fooling" the discriminator. To accomplish this task, it utilizes a series of transposed convolutional layers to progressively upscale the input vector into a full-sized image.

The **Discriminator** acts as a critic that tries to differentiate between real images (from the dataset) and fake images (produced by the generator). It uses standard convolutional layers to downsample the input image and makes a decision on its authenticity.

During training the generator tries to maximize the error rate of the discriminator (by improving the quality of the fake images), while the discriminator tries to minimize its own error rate. This adversarial process drives both networks to improve continuously, with the generator producing increasingly Monet-like painings and the discriminator becoming better at detecting subtleties between real and fake paintings. The training alternates between updating the discriminator with real and fake images, and updating the generator based on the feedback from the discriminator.

### Data handeling functions

In [ ]:
# Define the function to parse TFRecords
def _parse_function(proto):
    features = {
        'image': tf.io.FixedLenFeature([], tf.string),
    }
    parsed_features = tf.io.parse_single_example(proto, features)
    image = tf.io.decode_jpeg(parsed_features['image'], channels=3)
    image = tf.cast(image, tf.float32)
    image = (image - 127.5) / 127.5
    image = tf.reshape(image, [256, 256, 3])
    return image

# Load the dataset from TFRecords
def load_dataset(tfrecord_files, batch_size):
    dataset = tf.data.TFRecordDataset(tfrecord_files)
    dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

### GAN model

In [ ]:
# Generator model
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(8*8*1024, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 1024)),
        layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[256, 256, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Define loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

@tf.function
def train_step(images, generator, discriminator, generator_optimizer, discriminator_optimizer):
    noise = tf.random.normal([tf.shape(images)[0], 100])  # Use dynamic sizing for batch size

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss

def train(dataset, epochs, generator, discriminator, generator_optimizer, discriminator_optimizer):
    for epoch in range(epochs):
        start = time.time()
        
        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch, generator, discriminator, generator_optimizer, discriminator_optimizer)
            gen_loss_list.append(gen_loss)
            disc_loss_list.append(disc_loss)

        gen_loss_avg = tf.reduce_mean(gen_loss_list)
        disc_loss_avg = tf.reduce_mean(disc_loss_list)
        
        #print(f'Epoch {epoch+1}, gen_loss={gen_loss_avg:.4f}, disc_loss={disc_loss_avg:.4f}, time={time.time()-start:.2f} sec')

        if (epoch + 1) == 1 or (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}, gen_loss={gen_loss_avg:.4f}, disc_loss={disc_loss_avg:.4f}, time={time.time()-start:.2f} sec')
            generate_and_save_images(generator, epoch + 1, seed)

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(8, 8))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        img = (predictions[i, :, :, :] * 127.5 + 127.5).numpy().astype('uint8')
        plt.imshow(img)
        plt.axis('off')

    plt.show()

BATCH_SIZE = 32
tfrecord_files = [
    Input + '/monet_tfrec/monet00-60.tfrec',
    Input + '/monet_tfrec/monet04-60.tfrec',
    Input + '/monet_tfrec/monet08-60.tfrec',
    Input + '/monet_tfrec/monet12-60.tfrec',
    Input + '/monet_tfrec/monet16-60.tfrec'
]
dataset = load_dataset(tfrecord_files, BATCH_SIZE)

generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

num_examples_to_generate = 4
seed = tf.random.normal([num_examples_to_generate, 100])

# Start Training
train(dataset, 1500, generator, discriminator, generator_optimizer, discriminator_optimizer)


In [ ]:
def generate_and_save_images(model, total_images, directory, batch_size=10):
    if not os.path.exists(directory):
        os.makedirs(directory)  # Ensure the directory exists

    for batch_start in range(0, total_images, batch_size):
        batch_end = min(batch_start + batch_size, total_images)
        num_images = batch_end - batch_start
        noise = tf.random.normal([num_images, 100])  # Generate different noises for each batch
        predictions = model(noise, training=False)

        for i, img in enumerate(predictions):
            img = (img * 127.5 + 127.5).numpy().astype('uint8')
            image = Image.fromarray(img)
            image.save(f"{directory}/image_{batch_start+i+1:03d}.jpg", 'JPEG')  # Save each image as JPEG

            # Directory where images will be saved
output_dir = '../images'

# After training, generate and save 100 images in JPEG format
generate_and_save_images(generator, 7000, output_dir)

In [ ]:
# Display some generated images
path = '../images'
display_images(path)

### Conclusion

In this assignment, we have seen that it is possible to build an GAN model that can produce Monet-style paintings. However, with the training time used in this project, it not difficult for a human to distinguish a real from a fake Monet. The GAN gets the colors right, but the figurative aspects of the paintings are wrong or missing. More training time could perhaps have improved the figurative aspects of the paintings. But it is far from certain that the GAN would have been able to learn these aspects of the Monet-style. 

We chose to create a DCGAN that created the images from scratch (so to speak) rather than training a cycle GAN that could style pictures. It would have been interesting to compare the two approaches, but that falls outside the scope of this mini-project.  

In [ ]:
import zipfile

sdir = "../images/"
zfile = "images.zip"
with zipfile.ZipFile("images.zip","w") as zipf:
    for file in os.listdir("../images/"):
        fpath = os.path.join(sdir,file)
        zipf.write(fpath, os.path.basename(file))